# Libraries

In [ ]:
# !pip install scikit-learn
# !pip install spacy
# !python -m spacy download en_core_web_sm
# !pip install torchtext==0.6.0
# !pip install transformers sentence-transformers
# !pip install tqdm

In [1]:
from joblib import load
import pandas as pd
import numpy as np
# import spacy
# from torchtext import data
# from torchtext.vocab import GloVe
# import torch
# import random
# from transformers import DebertaTokenizer, DebertaModel
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
# from sklearn.metrics import classification_report

# Load models and data

In [2]:
test_data = pd.read_csv('test.csv')
svm_sBERT = load('models/svm_sBERT.joblib')

# Process data

## Initialization

In [3]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

### Sentence Embedding

In [4]:
# Function to apply model encoding with a progress bar
def encode_with_progress(series):
    return series.astype(str).progress_apply(model.encode)

# Function to combine premise and hypothesis
def combine_embeddings(embedding1, embedding2):
    # Element-wise difference
    difference = np.subtract(embedding1, embedding2)
    
    # Element-wise product
    product = np.multiply(embedding1, embedding2)
    
    # Concatenate features
    combined_embedding = np.concatenate((embedding1, embedding2, difference, product))
    
    return combined_embedding

# Initialize tqdm within the pandas apply
tqdm.pandas()

# Embedding the data
test_data['embedding_premise'] = encode_with_progress(test_data['premise'])
test_data['embedding_hypothesis'] = encode_with_progress(test_data['hypothesis'])

# Combining the data
test_data['combined_embedding'] = test_data.apply(lambda x: combine_embeddings(x['embedding_premise'], x['embedding_hypothesis']), axis=1)

  0%|          | 0/3302 [00:00<?, ?it/s]

  0%|          | 0/3302 [00:00<?, ?it/s]

# Predict

In [ ]:
y_pred_sBERT = svm_sBERT.predict(test_data['combined_embedding'].tolist())

# Save result

In [ ]:
df = pd.DataFrame({"prediction": y_pred_sBERT})
df.to_csv('Group_9_A.csv', index=False)